Importing the libraries

In [4]:
import pandas as pd
import sqlite3 as sql

Make a connection to the server

In [5]:
conn = sql.connect('/content/drive/MyDrive/sql-murder-mystery.db')

In [6]:
print(conn)

Taking a look at all the tables

In [9]:
query_1 = '''
SELECT name
  FROM sqlite_master
 where type = 'table'
'''
pd.set_option('display.max_colwidth', None)
pd.read_sql_query(query_1, conn)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [8]:
print(query_1)


SELECT name 
  FROM sqlite_master
 where type = 'table'



Retreiving the Crime Scene Report

In [10]:
query_2 = '''
SELECT * FROM crime_scene_report
WHERE city = "SQL City"
ORDER BY date;
'''
pd.set_option('display.max_colwidth', None)
pd.read_sql_query(query_2, conn)

,date,type,description,city
0,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
1,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
2,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City
3,20180103,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual.",SQL City
4,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
5,20180115,assault,Report Not Found,SQL City
6,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
7,20180215,murder,REDACTED REDACTED REDACTED,SQL City
8,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


Using the security footage finding the house address of the witnesses

In [11]:
query_3 = '''
SELECT *
FROM Person
WHERE address_street_name ='Northwestern Dr'
ORDER BY address_number desc;
'''
pd.read_sql_query(query_3, conn)[:1]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


The first witness is Morty Schapiro

In [17]:
query_4 = '''
SELECT *
FROM Person
WHERE address_street_name ='Franklin Ave'
ORDER BY name;
'''
pd.read_sql_query(query_4, conn)[1:2]

,id,name,license_id,address_number,address_street_name,ssn
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143


The second witness is Annabel Miller

Getting the data from the interview of the witnesses

In [21]:
query_5 = '''
SELECT * FROM interview
WHERE person_id = 14887
'''
pd.read_sql_query(query_5, conn)   # Getting details from Morty Schaphiro

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [22]:
query_6 = '''
SELECT * FROM interview
WHERE person_id = 16371
'''
pd.read_sql_query(query_6, conn)   #Getting details from Annabel Miller

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Checking the gym database

In [30]:
query_7 = '''
SELECT * FROM get_fit_now_member
WHERE membership_status = 'gold' and id like '48Z%'
'''
pd.read_sql_query(query_7, conn)  # Going through the gym database

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Checking the car details

In [34]:
query_8 = '''
SELECT * FROM drivers_license
WHERE plate_number like '%H42W%'
'''
pd.read_sql_query(query_8, conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [38]:
query_9 = '''
SELECT p.name, p.id, plate_number FROM person p
LEFT JOIN drivers_license d on d.id = p.license_id
WHERE p.id in (67318, 28819) and d.plate_number like '%H42W%'
'''
pd.read_sql_query(query_9, conn)

,name,id,plate_number
0,Jeremy Bowers,67318,0H42W2


From this information the killer is Jeremy Bowers

Further Findings

In [42]:
query_10 = '''
SELECT * FROM interview
WHERE person_id = 67318
'''
pd.read_sql_query(query_10, conn)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


Checking the events to find the actual killer

In [51]:
query_11 = '''
SELECT p.id, p.name, d.hair_color, d.height, d.gender, i.annual_income, d.car_make, d.car_model, f.event_name
FROM person p
LEFT JOIN drivers_license d on d.id = p.license_id
LEFT JOIN income i on i.ssn = p.license_id
LEFT JOIN facebook_event_checkin f on p.id = f.person_id
WHERE d.hair_color = 'red' and height BETWEEN 65 and 67 and gender = 'female' and car_make = 'Tesla'
and event_name = 'SQL Symphony Concert' and date between '20171201' and '20171231'
ORDER BY i.annual_income desc
'''
pd.read_sql_query(query_11, conn)

,id,name,hair_color,height,gender,annual_income,car_make,car_model,event_name
0,99716,Miranda Priestly,red,66,female,None,Tesla,Model S,SQL Symphony Concert
1,99716,Miranda Priestly,red,66,female,None,Tesla,Model S,SQL Symphony Concert
2,99716,Miranda Priestly,red,66,female,None,Tesla,Model S,SQL Symphony Concert


So the murderer was Miranda Priestly who hired Jeremy Bowers!!!